#### This particular notebook details the combined preprocessing pipeline for our Xenium Dataset 1 (integrated rep 1 and 2) data

#### Required input files:

* Raw Xenium Dataset 1 290 IntReps1and2 data object (availabile via FigShare) OR raw data files for Xenium Dataset 1 290 Rep1 and Xenium Dataset 1 290 Rep2 (available via GEO)

This notebook starts with the raw GEO files

Note: Rep1 is occasionally referred to as the 'Sept' run, while Rep2 is occasionally referred to as the 'Nov' run.

Environment: Please create and activate the conda environment provided in harmony_integration_env.yaml before running this notebook

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import gzip
import anndata

# New addition for harmony_integration_env.yaml compared to default_env.yaml
import harmonypy as hm

### Load Data

### Nov / Rep2 run

Load in essential data object files and store df within the object obs slot

In [ ]:
adata_Nov = sc.read_10x_h5(
    filename="/path/cell_feature_matrix.h5"
)

In [ ]:
# We can unzip cells.csz.gz to obtain cells.csv

In [ ]:
df_Nov = pd.read_csv(
    "/path/cells.csv"
)

In [ ]:
df_Nov.set_index(adata_Nov.obs_names, inplace=True)
adata_Nov.obs = df_Nov.copy()

In [ ]:
# Store coordinates within obsm spatial
adata_Nov.obsm["spatial"] = adata_Nov.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

In [ ]:
adata_Nov.obsm["spatial"]

In [ ]:
adata_Nov.obs

# 424,230 cells

In [ ]:
adata_Nov

#### Extra data object edits

Rename a few metadata column names (to match previous notebooks) and add a column for batch

In [ ]:
adata_Nov.obs.rename(columns={'cell_area': 'cell_area_um2'}, inplace=True)

adata_Nov.obs['batch'] = 'Nov'

Continue -- Add cell_id back as row names and as an individual column. 

Additionally, rename each cell_id value to start with "Nov-" to differentiate the Xenium runs

In [ ]:
# Rename the cell_id values to start with "Nov-" to denote the appropriate Xenium run
adata_Nov_metadata["cell_id"] = "Nov-" + adata_Nov_metadata["cell_id"]


### Add the cell_ids as row names, but keep as a column

# Copy 'cell_id' to a new column
adata_Nov_metadata['cell_id2'] = adata_Nov_metadata['cell_id']

# Set 'cell_id2' as the index
adata_Nov_metadata.set_index('cell_id2', inplace=True)

# Remove the name of the index
adata_Nov_metadata.index.name = None

In [ ]:
# Add the modified metadata back into the adata_Nov object and view

adata_Nov.obs = adata_Nov_metadata

adata_Nov.obs

Offset x_centroid (just for 1 of the 2 datasets (choosing Nov run) so that we can plot them in the same space) and store coordinates (centroid x and y values) within the object obsm spatial slot

In [ ]:
# UPDATE -- Adding / offsetting x_centroid by 15,000 (since adata_Sept.obs['x_centroid'].max() = ~10,000)
adata_Nov.obs['x_centroid'] += 15000

In [ ]:
# Store UPDATED coordinates within obsm spatial
adata_Nov.obsm["spatial"] = adata_Nov.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

View the newly formed and edited data object

In [ ]:
adata_Nov.obs

# 424,230 cells

In [ ]:
adata_Nov

### Sept / Rep1 run

Load in essential data object files and store df within the object obs slot

In [ ]:
adata_Sept = sc.read_10x_h5(
    filename="/path/cell_feature_matrix.h5"
)

In [ ]:
# We need to unzip cells.csz.gz to obtain cells.csv -- Already done, so I'm commenting out / deleting this section

In [ ]:
df_Sept = pd.read_csv(
    "/path/cells.csv"
)

In [ ]:
df_Sept.set_index(adata_Sept.obs_names, inplace=True)
adata_Sept.obs = df_Sept.copy()

View the newly formed and edited data object

In [ ]:
adata_Sept.obs

# 434,666 cells

In [ ]:
adata_Sept

#### Extra data object edits

Rename a few metadata column names (to match previous notebooks) and add a column for batch

In [ ]:
adata_Sept.obs.rename(columns={'cell_area': 'cell_area_um2'}, inplace=True)

adata_Sept.obs['batch'] = 'Sept'

Continue -- Add cell_id back as row names and as an individual column. 

Additionally, rename each cell_id value to start with "Sept-" to differentiate the Xenium runs

In [ ]:
# Rename the cell_id values to start with "Sept-" to denote the appropriate Xenium run
adata_Sept_metadata["cell_id"] = "Sept-" + adata_Sept_metadata["cell_id"]


### Add the cell_ids as row names, but keep as a column

# Copy 'cell_id' to a new column
adata_Sept_metadata['cell_id2'] = adata_Sept_metadata['cell_id']

# Set 'cell_id2' as the index
adata_Sept_metadata.set_index('cell_id2', inplace=True)

# Remove the name of the index
adata_Sept_metadata.index.name = None

In [ ]:
# Add the modified metadata back into the adata_Sept object and view

adata_Sept.obs = adata_Sept_metadata

adata_Sept.obs

Note: No need to offset x_centroid here since we already did it for the Nov run.

View the newly formed and edited data object

In [ ]:
adata_Sept.obs

# 434,666 cells

In [ ]:
adata_Sept

In [ ]:
adata_Sept.obs['x_centroid'].max()

#### Concluding Notes (to inform concatenating)
* All of the columns within the obs slot are the same. The row names (cell_id) have overlap -- but we've added identifier strings (Nov- or Sept-) to make them unique
* Obsm spatial has the coordinates stored in an array -- different but overlapping. We've offset the x_centroid values for adata_Nov by 15,000
* Var gene_ids, feature_types, and genome are the same across objects, since the same gene panel was used

### Save data objects before continuing

These adata objects can be loaded in for future versions of this pipeline. Note: They should only be used for this concat pipeline, due to particular formatting decisions (like adding the batch information and offsetting some spatial coordinates).

In [ ]:
adata_Nov.write_h5ad('/path/XeniumNovData_initialformatforconcat.h5ad')
adata_Sept.write_h5ad('/path/XeniumSeptData_initialformatforconcat.h5ad')

### Calculate Quality Control Metrics (for both data objects)

Calculate the quality control metrics on the anndata.AnnData using scanpy.pp.calculate_qc_metrics

Reminder -- This overrides Xeniun's original total_counts value, but I'm choosing to let this happen. The original counts value represents gene expression features, control probes, control codewords, and unassigned codewords. The new version includes (solely) the gene expression feature count

In [ ]:
sc.pp.calculate_qc_metrics(adata_Sept, percent_top=(10, 20, 50, 150), inplace=True)

sc.pp.calculate_qc_metrics(adata_Nov, percent_top=(10, 20, 50, 150), inplace=True)

The percentage of control probes and control codewords can be calculated from adata.obs

In [ ]:
cprobes_Sept = (
    adata_Sept.obs["control_probe_counts"].sum() / adata_Sept.obs["total_counts"].sum() * 100
)
cwords_Sept = (
    adata_Sept.obs["control_codeword_counts"].sum() / adata_Sept.obs["total_counts"].sum() * 100
)

print("adata_Sept")
print(f"Negative DNA probe count % : {cprobes_Sept}")
print(f"Negative decoding count % : {cwords_Sept}")
print("\n")

cprobes_Nov = (
    adata_Nov.obs["control_probe_counts"].sum() / adata_Nov.obs["total_counts"].sum() * 100
)
cwords_Nov = (
    adata_Nov.obs["control_codeword_counts"].sum() / adata_Nov.obs["total_counts"].sum() * 100
)

print("adata_Nov")
print(f"Negative DNA probe count % : {cprobes_Nov}")
print(f"Negative decoding count % : {cwords_Nov}")

Next we plot the distribution of total transcripts per cell, unique transcripts per cell, area of segmented cells and the ratio of nuclei area to their cells

In [ ]:
## Sept

fig, axs = plt.subplots(1, 4, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata_Sept.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    adata_Sept.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)


axs[2].set_title("Area of segmented cells")
sns.histplot(
    adata_Sept.obs["cell_area_um2"],
    kde=False,
    ax=axs[2],
)

axs[3].set_title("Nucleus ratio")
sns.histplot(
    adata_Sept.obs["nucleus_area"] / adata_Sept.obs["cell_area_um2"],
    kde=False,
    ax=axs[3],
)

In [ ]:
## Nov

fig, axs = plt.subplots(1, 4, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata_Nov.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    adata_Nov.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)


axs[2].set_title("Area of segmented cells")
sns.histplot(
    adata_Nov.obs["cell_area_um2"],
    kde=False,
    ax=axs[2],
)

axs[3].set_title("Nucleus ratio")
sns.histplot(
    adata_Nov.obs["nucleus_area"] / adata_Nov.obs["cell_area_um2"],
    kde=False,
    ax=axs[3],
)

Filter the cells based on the minimum number of counts required using scanpy.pp.filter_cells. Filter the genes based on the minimum number of cells required with scanpy.pp.filter_genes. The parameters for the both were specified based on the plots above. They were set to filter out the cells and genes with minimum counts and minimum cells respectively.

Other filter criteria might be cell area, DAPI signal or a minimum of unique transcripts.

Squidpy tutorial filtering examples: 
sc.pp.filter_cells(adata, min_counts=10);
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
## Our filtering schema
# Note: Transcripts that didn't pass 10x Genomics's QC have already been filtered out

## Sept
adata_Sept_filtered = adata_Sept

# filter out cells with <50 counts and <10 genes
sc.pp.filter_cells(adata_Sept_filtered, min_counts=50)
sc.pp.filter_cells(adata_Sept_filtered, min_genes=10)

# filter out genes that have <1 count and are detected in <10 cells
sc.pp.filter_genes(adata_Sept_filtered, min_counts=1)
sc.pp.filter_genes(adata_Sept_filtered, min_cells=10)


## Nov
adata_Nov_filtered = adata_Nov

# filter out cells with <50 counts and <10 genes
sc.pp.filter_cells(adata_Nov_filtered, min_counts=50)
sc.pp.filter_cells(adata_Nov_filtered, min_genes=10)

# filter out genes that have <1 count and are detected in <10 cells
sc.pp.filter_genes(adata_Nov_filtered, min_counts=1)
sc.pp.filter_genes(adata_Nov_filtered, min_cells=10)

#### Compare filtered objects

In [ ]:
adata_Sept_filtered.obs

# 313,940 total cells

In [ ]:
adata_Nov_filtered.obs

# 268,248 total cells

In [ ]:
# Nov -- 424,230 cells

# Sept -- 434,666 cells


## Total cells retained post-filtering (from 10x initial filtering step to our filtering step)
# Values pasted from above

print(f"Sept % of cells maintained post my filtering: {(313940 / 434666) * 100}")

print(f"Nov % of cells maintained post my filtering: {(268248 / 424230) * 100}")

In [ ]:
## Total transcripts retained post-filtering (from 10x initial filtering step to our filtering step)

total_counts_sum_Sept = adata_Sept.obs['total_counts'].sum()
total_counts_sum_Sept_filtered = adata_Sept_filtered.obs['total_counts'].sum()

percent_maintained_Sept = ((total_counts_sum_Sept_filtered / total_counts_sum_Sept) * 100)

print("Sept % of transcripts (total_counts) maintained post my filtering:", percent_maintained_Sept)

total_counts_sum_Nov = adata_Nov.obs['total_counts'].sum()
total_counts_sum_Nov_filtered = adata_Nov_filtered.obs['total_counts'].sum()

percent_maintained_Nov = ((total_counts_sum_Nov_filtered / total_counts_sum_Nov) * 100)

print("Nov % of transcripts (total_counts) maintained post my filtering:", percent_maintained_Nov)

In [ ]:
## Calculate min, mean, and max values of adata_Sept_filtered

total_counts_minvalue = adata_Sept_filtered.obs["total_counts"].min()
total_counts_mean = adata_Sept_filtered.obs["total_counts"].mean()
total_counts_maxvalue = adata_Sept_filtered.obs["total_counts"].max()

n_genes_by_counts_minvalue = adata_Sept_filtered.obs["n_genes_by_counts"].min()
n_genes_by_counts_mean = adata_Sept_filtered.obs["n_genes_by_counts"].mean()
n_genes_by_counts_maxvalue = adata_Sept_filtered.obs["n_genes_by_counts"].max()

cell_area_um2_minvalue = adata_Sept_filtered.obs["cell_area_um2"].min()
cell_area_um2_mean = adata_Sept_filtered.obs["cell_area_um2"].mean()
cell_area_um2_maxvalue = adata_Sept_filtered.obs["cell_area_um2"].max()

print("adata_Sept_filtered\n")

print(f"Min value of total_counts : {total_counts_minvalue}")
print(f"Mean of total_counts : {total_counts_mean}")
print(f"Max value of total_counts : {total_counts_maxvalue}\n")

print(f"Min value of n_genes_by_counts : {n_genes_by_counts_minvalue}")
print(f"Mean of n_genes_by_counts : {n_genes_by_counts_mean}")
print(f"Max value of n_genes_by_counts : {n_genes_by_counts_maxvalue}\n")

print(f"Min value of cell area (um2) : {cell_area_um2_minvalue}")
print(f"Mean of cell area (um2) : {cell_area_um2_mean}")
print(f"Max value of cell area (um2) : {cell_area_um2_maxvalue}\n")

## Calculate min, mean, and max values of adata_Nov_filtered

total_counts_minvalue = adata_Nov_filtered.obs["total_counts"].min()
total_counts_mean = adata_Nov_filtered.obs["total_counts"].mean()
total_counts_maxvalue = adata_Nov_filtered.obs["total_counts"].max()

n_genes_by_counts_minvalue = adata_Nov_filtered.obs["n_genes_by_counts"].min()
n_genes_by_counts_mean = adata_Nov_filtered.obs["n_genes_by_counts"].mean()
n_genes_by_counts_maxvalue = adata_Nov_filtered.obs["n_genes_by_counts"].max()

cell_area_um2_minvalue = adata_Nov_filtered.obs["cell_area_um2"].min()
cell_area_um2_mean = adata_Nov_filtered.obs["cell_area_um2"].mean()
cell_area_um2_maxvalue = adata_Nov_filtered.obs["cell_area_um2"].max()

print("adata_Nov_filtered\n")

print(f"Min value of total_counts : {total_counts_minvalue}")
print(f"Mean of total_counts : {total_counts_mean}")
print(f"Max value of total_counts : {total_counts_maxvalue}\n")

print(f"Min value of n_genes_by_counts : {n_genes_by_counts_minvalue}")
print(f"Mean of n_genes_by_counts : {n_genes_by_counts_mean}")
print(f"Max value of n_genes_by_counts : {n_genes_by_counts_maxvalue}\n")

print(f"Min value of cell area (um2) : {cell_area_um2_minvalue}")
print(f"Mean of cell area (um2) : {cell_area_um2_mean}")
print(f"Max value of cell area (um2) : {cell_area_um2_maxvalue}\n")

Visualize genes with the highest expression levels

In [ ]:
sc.pl.highest_expr_genes(adata_Sept_filtered, n_top=20, )

In [ ]:
sc.pl.highest_expr_genes(adata_Nov_filtered, n_top=20, )

### Concatenate the 2 objects into 1 anndata object

With concat(), AnnData objects can be combined via a composition of two operations: concatenation and merging.

* Concatenation is when we keep all sub elements of each object, and stack these elements in an ordered way.
* Merging is combining a set of collections into one resulting collection which contains elements from the objects.

In [ ]:
# Concatenate along the observations axis (axis=0)
concat_filtered_ad = anndata.concat([adata_Sept_filtered, adata_Nov_filtered], axis=0)

In [ ]:
concat_filtered_ad

In [ ]:
concat_filtered_ad.obs

# 582,188 cells (313,940 from Sept and 268,248 from Nov)

In [ ]:
concat_filtered_ad.obsm['spatial']

In [ ]:
sc.pl.scatter(concat_filtered_ad, x='x_centroid', y='y_centroid', color='batch', show=True)

### Now continue with analysis on concatenated object

Make a copy of the original raw counts (post-filtering; pre-normalization)

In [ ]:
concat_filtered_ad.layers['raw_counts'] = concat_filtered_ad.X.copy() # Make a copy
concat_filtered_ad

In [ ]:
concat_filtered_ad.obs

### Save object

This is post-filtering and pre-normalization

In [ ]:
concat_filtered_ad.write_h5ad('/path/DataObjects_withoutUMAP/Xenium_concatSeptNov_filtered_pre-normalization.h5ad')

### Continue with analysis

Normalize counts per cell using scanpy.pp.normalize_total.

Logarithmize, do principal component analysis, compute a neighborhood graph of the observations using scanpy.pp.log1p, scanpy.pp.pca and scanpy.pp.neighbors respectively.

Use scanpy.tl.umap to embed the neighborhood graph of the data and cluster the cells into subgroups employing scanpy.tl.leiden.

In [ ]:
sc.pp.normalize_total(concat_filtered_ad, inplace=True)
sc.pp.log1p(concat_filtered_ad)
# Save log_normalized_counts as a layer
concat_filtered_ad.layers['log_normalized_counts']=concat_filtered_ad.X

Calculate and plot the top highly variable genes

In [ ]:
sc.pp.highly_variable_genes(concat_filtered_ad, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(concat_filtered_ad)

Regress out unwanted sources or variation and scale data

#### Note: All objects in this notebook will undergo this step

In [ ]:
# Here, we're just going to regress out total_counts and n_genes_by_counts

sc.pp.regress_out(concat_filtered_ad, ["total_counts","n_genes_by_counts"])

In [ ]:
sc.pp.scale(concat_filtered_ad, max_value=10)

Run PCA and plot PCA variance ratio

In [ ]:
sc.pp.pca(concat_filtered_ad, svd_solver='arpack')

In [ ]:
sc.pl.pca(concat_filtered_ad, color=['EPCAM','COL3A1','CD69'])

In [ ]:
# Plot the PCA variance ratio
sc.pl.pca_variance_ratio(concat_filtered_ad, log=True)

In [ ]:
# Plot the PCA variance ratio
sc.pl.pca_variance_ratio(concat_filtered_ad, n_pcs = 50, log=True)

In [ ]:
concat_filtered_ad

### Save object

This is post-filtering and post-normalization. Immediately before computing neighbor graph and plotting UMAP.

In [ ]:
concat_filtered_ad.write_h5ad('/path/DataObjects_withoutUMAP/Xenium_concatSeptNov_filtered_post-normalization.h5ad')

### Run integration with harmony

(Even though these datasets are replicates, it's still good to run Harmony just in case)

In [ ]:
# Prepare metadata and PCA

meta_data = concat_filtered_ad.obs
data_mat = concat_filtered_ad.obsm['X_pca']

In [ ]:
# Run harmony

ho = hm.run_harmony(data_mat, meta_data, 'batch')

In [ ]:
# Mapping back the result to the adata object

concat_filtered_ad.obsm['X_pca'] = ho.Z_corr.T

In [ ]:
# Save object

concat_filtered_ad.write_h5ad('/path/DataObjects_withoutUMAP/Xenium_concatSeptNov_filtered_post-normalization_post-harmony.h5ad')

In [ ]:
concat_filtered_ad

In [ ]:
concat_filtered_ad.obsm['X_pca']

In [ ]:
# Re-plot PCA visualization
sc.pl.pca(concat_filtered_ad, color=['EPCAM','COL3A1','CD69'])

In [ ]:
# Re-plot the second PCA variance ratio
sc.pl.pca_variance_ratio(concat_filtered_ad, log=True)

In [ ]:
# Re-plot the second PCA variance ratio
sc.pl.pca_variance_ratio(concat_filtered_ad, n_pcs= 50, log=True)

### Compute neighbor graph and plot UMAP

In [ ]:
sc.settings.figdir = '/path/UMAP_pngs/'

#### Parameter descriptions:
n_neighbors
* A value between 2 and 100, representing the number of neighboring data points used for manifld approximation. Larger values give a manifold with a more global view of the dataset, while smaller values preserve more of the local structures.
* Default value is 15

n_pcs
* Use this many PCs
* Default value is None
* Choose the value after the elbow, not before

min_dist
* The minimum distance between two points in the UMAP embedding.
* Default value is 0.05

spread
* A scaling factor for distance between embedded points.
* Default value is 1.0

Helpful resource: https://smorabit.github.io/blog/2020/umap/

#### Testing v2

In [ ]:
concat_filtered_ad_v2 = concat_filtered_ad

In [ ]:
sc.pp.neighbors(concat_filtered_ad_v2, n_neighbors=10, n_pcs=30)
sc.tl.umap(concat_filtered_ad_v2, min_dist=0.02, spread=1.5)
sc.tl.leiden(concat_filtered_ad_v2)

In [ ]:
sc.pl.umap(
    concat_filtered_ad_v2,
    color=[
        "total_counts",
        "n_genes_by_counts",
        "leiden",
    ],
    wspace=0.4,
    save = '_v2_Xeniumconcat.png',
)

In [ ]:
sc.pl.umap(
    concat_filtered_ad_v2,
    color="batch",
    wspace=0.4,
    save="_v2_Xeniumconcat_splitbybatch.png",
)

In [ ]:
# Save object with UMAP
concat_filtered_ad_v2.write_h5ad('/path/DataObjects_withUMAP/Xeniumconcat_umapv2_231210.h5ad')

### Generate umap without running harmony (for batch effect reference)

In [ ]:
## Read in file (no harmony v2 umap)

concat_filtered_ad_nhv2 = sc.read_h5ad('/path/DataObjects_withoutUMAP/Xenium_concatSeptNov_filtered_post-normalization_231210.h5ad')

# View
concat_filtered_ad_nhv2

In [ ]:
sc.pp.neighbors(concat_filtered_ad_nhv2, n_neighbors=10, n_pcs=30)
sc.tl.umap(concat_filtered_ad_nhv2, min_dist=0.02, spread=1.5)
sc.tl.leiden(concat_filtered_ad_nhv2)

In [ ]:
sc.pl.umap(
    concat_filtered_ad_nhv2,
    color=[
        "total_counts",
        "n_genes_by_counts",
        "leiden",
    ],
    wspace=0.4,
    save="_v2_Xeniumconcat_withoutharmony.png",
)

In [ ]:
sc.pl.umap(
    concat_filtered_ad_nhv2,
    color="batch",
    wspace=0.4,
    save="_v2_Xeniumconcat_withoutharmony_splitbybatch.png",
)

In [ ]:
# Save object with UMAP
concat_filtered_ad_nhv2.write_h5ad('/path/DataObjects_withUMAP/Xeniumconcat_withoutharmony_umapv2_231210.h5ad')